In [60]:
import numpy as np 
import pandas as pd
import torch

In [61]:
cv=torch.load('dataset/course_nm_tensor.pt')

In [62]:
cvnpy=cv.detach().numpy()
cvnpy[:4]

array([[ 0.00144092, -0.01703043,  0.00412345, ...,  0.0865251 ,
        -0.05929217,  0.00758869],
       [-0.00358773, -0.01746405, -0.12736031, ...,  0.02512024,
        -0.01919973,  0.01910394],
       [-0.00345605, -0.03809951, -0.0692881 , ...,  0.03877716,
        -0.03475577, -0.03682799],
       [ 0.0022921 , -0.07450938, -0.00588197, ...,  0.00472871,
        -0.03109727, -0.00817915]], dtype=float32)

In [63]:
#group by whether it is 전공 
#같은 전공 내에서 진행

curridb=pd.read_csv('original_data/curriculumDB.txt')
coursedb=pd.read_csv('original_data/courseDB.txt')
ccdb=pd.read_csv('original_data/curri_courseDB.txt')

In [64]:
curridb.head(5)

,curr_id,the_year,subject_name,major_division,elec_num,ge_info
0,628b75c1920a8302ce7ea449535834a877720b79499426...,2019,건축도시시스템공학,1,138,10
1,e3e972226e56ed375c91a5f95e8c7f098e7bd3bfbac3e9...,2019,건축학,1,148,10
2,2ec8f0ea892df31db7f724bb6f5905e5a32a524ee7d33f...,2019,기후·에너지시스템공학,1,138,10
3,b4a6f11c5c2fdf083a2cc098db210bd7a91c1357a1164e...,2019,사이버보안,1,139,10
4,07902b1e387b5dcdd2ae8d9929576ac23aacd48c628d3a...,2019,식품공학,1,138,10


In [65]:
type(curridb.loc[0,'the_year'])
curridb.loc[0,'the_year']==2019
curridb.loc[3,'subject_name'].strip()=='사이버보안'
#print(curridb.loc[3,'subject_name'])

True

In [66]:
#subject_name, the_year정보를 바탕으로 구한다
subject_name='사이버보안'
the_year=2019

curr_id=curridb[(curridb['the_year']==the_year) & (curridb['subject_name'].apply(str.strip)==subject_name)]['curr_id']
curr_id

3    b4a6f11c5c2fdf083a2cc098db210bd7a91c1357a1164e...
Name: curr_id, dtype: object

In [67]:
ccdb.head(5)
type(ccdb['curr_id'].values)

np.array(ccdb.loc[0,'curr_id'])
str(curr_id.values[0])

'b4a6f11c5c2fdf083a2cc098db210bd7a91c1357a1164e01abae4e28d845b21a'

In [69]:
cvnpy.shape

(1035, 768)

In [78]:
from sklearn.metrics.pairwise import cosine_similarity
class Node:
    def __init__(self,course_id,standard,curr_id,course_vec):
        self.course_id=course_id 
        self.standard=standard 
        self.curr_id=curr_id 
        self.course_vec=course_vec

class RankMatrix:
    def __init__(self,nodes):
        self.nodes=nodes
        self.similarity_matrix=None
        self.run_cosine_similarity() 
        
    def run_cosine_similarity(self):
        course_vec=[node.course_vec for node in self.nodes] 
        self.similarity_matrix=cosine_similarity(course_vec,course_vec)
        
    def retrieve_similarity(self,course_id):
        course_ids=[i.course_id for i in self.nodes]
        course_standards=[i.standard for i in self.nodes]
        index_course=course_ids.index(course_id)
        return self.similarity_matrix[index_course],course_ids,course_standards
    
    
    def retrieve_top_10(self,course_ids,filter=''):
        #make it a node 
        def create_rank_node(course_id,sim,ccourse_id,stds):#ccourse_id는 비교 대상, course_id는 이번에 비교되는 애임 sim은 둘 사이 유사도임
            return {'course_id':course_id,'sim':sim,'ccourse_id':ccourse_id,'stds':stds}
        
        sim_list=[]
        for course in course_ids:
            sims,ids,stds=self.retrieve_similarity(course)
            for similarity,cid,std in zip(sims,ids,stds):
                if course==cid:
                    continue
                sim_list.append(create_rank_node(cid,similarity,course,std))
        
        result=merge_sort(sim_list)

        if filter=='전공':
            for node in result:
                if node['stds'].find('전공')==-1:#없으면
                    result.pop(result.index(node))
        elif filter=='융복합':
            for node in result:
                if node['stds'].find('융복합')==-1:#없으면
                    result.pop(result.index(node))
        else:
            for node in result:
                if node['stds'].find('전공')!=-1:#있으면
                    result.pop(result.index(node))
                if node['stds'].find('융복합')!=-1:#있으면
                    result.pop(result.index(node))
        return result[-10:]
    
r=RankMatrix([Node('1938','standard','dkfjsl',vec) for vec in np_1])
r.retrieve_similarity('1938')

(array([1.        , 0.39910215]), ['1938', '1938'], ['standard', 'standard'])

In [71]:
def merge(left, right):
    if len(left) == 0:
        return right

    if len(right) == 0:
        return left

    result = []
    index_left = index_right = 0

    while len(result) < len(left) + len(right):
        
        if left[index_left]['sim'] <= right[index_right]['sim']:
            result.append(left[index_left])
            index_left += 1
        else:
            result.append(right[index_right])
            index_right += 1

        # If you reach the end of either array, then you can
        # add the remaining elements from the other array to
        # the result and break the loop
        if index_right == len(right):
            result += left[index_left:]
            break

        if index_left == len(left):
            result += right[index_right:]
            break

    return result
def merge_sort(array):
    
    if len(array) < 2:
        return array

    midpoint = len(array) // 2
    
    return merge(
        left=merge_sort(array[:midpoint]),
        right=merge_sort(array[midpoint:]))

In [72]:
#currid를 바탕으로 사용자의 전공과목을 구한다 
#cold start 
#사용자의 전공과목과 전부 인접행렬을 만들고 
#해당 과목 인접행렬에서 가장 높은 것 N개를 구함


all_c=ccdb[ccdb['curr_id']==str(curr_id.values[0])]['course_id'].values

#종류 식별
st_cDB=pd.read_csv('original_data/standard_courseDB.txt')

#모든 학수번호마다 종류 파악
node_list=[] #하나의 교과과정표에 있는 과목의 리스트이다.
course_list=list(coursedb['course_id'])
for c in all_c:
    st_cDB_curr=st_cDB[st_cDB['curr_id']==str(curr_id.values[0])] #사용자의 교과과정표 내 수업에 대해
    st_cDB_cc=st_cDB_curr[st_cDB_curr['course_id']==c ]['standard_name'].values[0] #standard이름 파악
    index=course_list.index(c)#courseDB내 위치 파악해서 벡터와 연결
    node_list.append(Node(c,st_cDB_cc,str(curr_id.values[0]),cvnpy[index]))#course_id, standard, vector 로 된 노드를 만듦 

## test: 그냥 아무 노드 10개를 가지고 시도 

In [73]:
node_list[10].course_id

'11033'

In [74]:
r=RankMatrix(node_list)
r.retrieve_top_10([i.course_id for i in node_list[:10]])

[{'course_id': '10776', 'sim': 0.9871884, 'ccourse_id': '11256'},
 {'course_id': '38919', 'sim': 0.9873543, 'ccourse_id': '11256'},
 {'course_id': '11211', 'sim': 0.9876052, 'ccourse_id': '11095'},
 {'course_id': '11083', 'sim': 0.98782355, 'ccourse_id': '11074'},
 {'course_id': '11287', 'sim': 0.9882017, 'ccourse_id': '11254'},
 {'course_id': '11065', 'sim': 0.9884581, 'ccourse_id': '10023'},
 {'course_id': '11065', 'sim': 0.98846877, 'ccourse_id': '11095'},
 {'course_id': '10559', 'sim': 0.9890709, 'ccourse_id': '11254'},
 {'course_id': '11385', 'sim': 0.9911164, 'ccourse_id': '11254'},
 {'course_id': '10776', 'sim': 0.99141276, 'ccourse_id': '11254'}]

## st_cDB를 가지고 이제 종류별로 가져오기

In [76]:
j=[]
y=[]
g=[]
for node in node_list:
    if node.standard.find('전공')!=-1:
        j.append(node.course_id)
    elif node.standard.find('융복합')!=-1:
        y.append(node.course_id)
    else:
        g.append(node.course_id)

In [79]:
#전공, 융복합, 교양에 대해 각각 10개만 뽑는 코드 짜기 
r=RankMatrix(node_list)

rcj=r.retrieve_top_10(j,filter='전공')
rcy=r.retrieve_top_10(y,filter='융복합')
rcg=r.retrieve_top_10(g,filter='교양')
rcg

[{'course_id': '10776',
  'sim': 0.99141276,
  'ccourse_id': '11254',
  'stds': '큐브(콘텐츠)'},
 {'course_id': '11254',
  'sim': 0.99141276,
  'ccourse_id': '10776',
  'stds': '나눔리더십 외'},
 {'course_id': '10776',
  'sim': 0.9925674,
  'ccourse_id': '11385',
  'stds': '큐브(콘텐츠)'},
 {'course_id': '11385',
  'sim': 0.9925674,
  'ccourse_id': '10776',
  'stds': '융합기초'},
 {'course_id': '11067',
  'sim': 0.9933573,
  'ccourse_id': '11092',
  'stds': '스페인어'},
 {'course_id': '11092',
  'sim': 0.9933573,
  'ccourse_id': '11067',
  'stds': '일본어'},
 {'course_id': '10072',
  'sim': 0.99346644,
  'ccourse_id': '11164',
  'stds': '큐브(디자인)'},
 {'course_id': '11164',
  'sim': 0.99346644,
  'ccourse_id': '10072',
  'stds': '큐브(콘텐츠)'},
 {'course_id': '11287',
  'sim': 0.99483895,
  'ccourse_id': '11288',
  'stds': '큐브(콘텐츠)'},
 {'course_id': '11288',
  'sim': 0.99483895,
  'ccourse_id': '11287',
  'stds': '큐브(콘텐츠)'}]

In [ ]:
#DB에 넣기

# 여러개 전공에 대해서 추천하기

In [ ]:
class Recommender:
    def __init__(self,subjects):
        self.subjects=subjects 
        self.get_curr_of_subjects()
    def get_curri_of_subjects(self):
        for subject in self.subjects:
            
    def merge_curri_into_node_list(self):
        pass
    def recommend(self):
        return rcj,rcy,rcg